## Pricing Analysis - Moderate Approach

### Importing Libraries and Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("./Cleaned_data_pricing.csv", encoding="utf-8")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   customer_id              500 non-null    int64 
 1   cohort                   500 non-null    object
 2   price                    500 non-null    int64 
 3   price_name               500 non-null    object
 4   commercial_type          500 non-null    object
 5   invoicing_period_length  500 non-null    int64 
 6   next_invoice_at          500 non-null    object
 7   avg_3m_admin_bookings    500 non-null    int64 
 8   avg_3m_user_bookings     500 non-null    int64 
 9   city                     500 non-null    object
 10  city_id                  500 non-null    int64 
 11  specialization_name      500 non-null    object
 12  specialization_id        500 non-null    int64 
dtypes: int64(7), object(6)
memory usage: 50.9+ KB


"Recognized_mrr" should be understood here as recognized monthly reccuring revenue. 
I calculated it by dividing a price by invoicing period leng

In [5]:
df["recognized_mrr"] = df["price"] / df["invoicing_period_length"]

In [6]:
df.sample(5)

,customer_id,cohort,price,price_name,commercial_type,invoicing_period_length,next_invoice_at,avg_3m_admin_bookings,avg_3m_user_bookings,city,city_id,specialization_name,specialization_id,recognized_mrr
491,173,01/04/2020,1056,Jameda import jameda_gold_pro 1056.00 m12,jameda_gold_pro,12,10/04/2025 00:00,391,54,Frankfurt,1055,Zahnarzt,125,88.0
140,104,01/07/2022,29,GoldPro 29€ monthly (70% discount),jameda_gold_pro,1,22/02/2025 00:00,2,1,Fürstenstein,11793,Heilpraktiker,74,29.0
293,232,01/05/2020,1908,Jameda import jameda_platin 1908.00 m12,jameda_platin,12,02/05/2025 00:00,0,15,Berlin,1007,Internist,36,159.0
476,488,01/09/2022,1056,Jameda import jameda_gold_pro 1056.00 m12,jameda_gold_pro,12,13/11/2025 00:00,100,13,Weiden,14959,Zahnarzt,125,88.0
77,56,01/03/2024,99,GoldPro 99€ monthly,jameda_gold_pro,1,27/02/2025 00:00,19,8,Berlin,1007,Radiologe,44,99.0


### Standarizing values and defining formulas
I used Min-Max normalization to improve model fitting and make potential correlations more observable.

I also defined the Customer Engagement Score (CES) to measure customer activity. In the formula, admin bookings are weighted slightly lower than user bookings because user bookings may drive additional revenue. The Price-Usage Fit Score assesses how well a customer's payment aligns with their engagement. I included the 0.01 parameter to avoid division by zero errors.
The Churn Risk Score is primarily influenced by price, with engagement acting as a counterbalance—lower pricing or higher engagement reduces churn risk.

In [7]:
def min_max_scale(column):
    return (column - column.min()) / (column.max() - column.min())

df["normalized_admin_bookings"] = min_max_scale(df["avg_3m_admin_bookings"])
df["normalized_user_bookings"] = min_max_scale(df["avg_3m_user_bookings"])
df["normalized_revenue"] = min_max_scale(df["recognized_mrr"])

df["engagement_score"] = (1 * df["normalized_admin_bookings"]) + (1.25 * df["normalized_user_bookings"])
df["price_usage_fit"] = df["normalized_revenue"] / (df["engagement_score"] + 0.01)  # Avoid division by zero
df["churn_risk_score"] = (2 * df["price_usage_fit"]) - (0.5 * df["engagement_score"])

In [8]:
df.sample()

,customer_id,cohort,price,price_name,commercial_type,invoicing_period_length,next_invoice_at,avg_3m_admin_bookings,avg_3m_user_bookings,city,city_id,specialization_name,specialization_id,recognized_mrr,normalized_admin_bookings,normalized_user_bookings,normalized_revenue,engagement_score,price_usage_fit,churn_risk_score
281,365,01/02/2023,1908,Platin 1908€ annual,jameda_platin,12,01/02/2025 00:00,2,21,Hessen,7162,Orthopäde & Unfallchirurg,34,159.0,0.002581,0.057851,0.814433,0.074895,9.59345,19.149452


### Dividing in Quartiles 

I divided the Churn Risk Score into quartiles to categorize customers based on their likelihood of churn into four risk categories: Low, Medium, High and Critical

In [9]:
df["churn_risk_quartiles"] = pd.qcut(df["churn_risk_score"], q=4, labels=["Low", "Medium", "High", "Critical"])

### Holdplace for further EDA - do some graphs, check the distribution etc. 

### Optimized Price and New Churn Risk Score Calculated

The value assigned to medium_churn_threshold represents the highest churn risk score within the Low segment, meaning it equals to the customer with the highest churn likelihood still classified as "Low" risk. This basically sets the cap on that threshold, ensuring optimized pricing won't raise churn above its level.

In [10]:
low_churn_threshold = df[df["churn_risk_quartiles"] == "Low"]["churn_risk_score"].max()
low_churn_threshold

5.531153474387166

I've added minimal and maximal revenue/recognized mrr to be able to later map the optimized price from normalized values back to absolute revenue figures

In [11]:
df["normalized_engagmnet"] = min_max_scale(df["engagement_score"])
min_revenue, max_revenue = df["recognized_mrr"].min(), df["recognized_mrr"].max()

### Computing Optimal Price
I adjusted prices for customers in the Low churn risk segments while keeping others unchanged. The maximum price is capped at 195 (Platinum Package). The threshold for price increases is set at a churn risk score of 5.53115 (described before), which corresponds to the last customer in the Medium risk category. If the calculated price exceeds $195, the function caps it and recalculates the churn risk score.

In [12]:
def compute_optimal_price(row):
    if row["churn_risk_quartiles"] not in ["Low"] or row["recognized_mrr"] == 1:
        return row["recognized_mrr"], row["churn_risk_score"]

    engagement_score_normalized = row["normalized_engagmnet"]
    target_price_usage_fit = (low_churn_threshold + (0.5 * engagement_score_normalized)) / 2
    new_normalized_revenue = target_price_usage_fit * (engagement_score_normalized + 0.01)
    new_price = (new_normalized_revenue * (max_revenue - min_revenue)) + min_revenue

    if new_price > 195:
        new_price = 195
        capped_normalized_revenue = (new_price - min_revenue) / (max_revenue - min_revenue)
        new_normalized_price_usage_score = capped_normalized_revenue / (engagement_score_normalized + 0.01)
        new_churn_risk_score = (2 * new_normalized_price_usage_score) - (0.5 * engagement_score_normalized)
    else:
        new_churn_risk_score = low_churn_threshold

    new_price = max(new_price, row["recognized_mrr"])
    new_churn_risk_score = max(new_churn_risk_score, row["churn_risk_score"])

    return round(new_price, 2), round(new_churn_risk_score, 4)

df[["optimized_price", "new_churn_risk_score"]] = df.apply(
    lambda row: compute_optimal_price(row), axis=1, result_type="expand")

In [13]:
df.sort_values(["new_churn_risk_score"]).head(1)

,customer_id,cohort,price,price_name,commercial_type,invoicing_period_length,next_invoice_at,avg_3m_admin_bookings,avg_3m_user_bookings,city,...,normalized_admin_bookings,normalized_user_bookings,normalized_revenue,engagement_score,price_usage_fit,churn_risk_score,churn_risk_quartiles,normalized_engagmnet,optimized_price,new_churn_risk_score
38,142,01/10/2024,1,GoldPro 1€ monthly,jameda_gold_pro,1,28/02/2025 00:00,290,19,Zweibrücken,...,0.374194,0.052342,0.0,0.439621,0.0,-0.21981,Low,0.267903,1.0,-0.21981


### Dividing new churn risk into quartiles - to do if needed

### Analysis on viability and impact of the price change
I selected customers in the Low churn risk segments and calculated their total current revenue (price change only involved them). Then, I calculated the total revenue based on the optimized price. I sorted these customers by new churn risk score in descending order for removing higher-risk customers first.

Next, the function iteratively removes customers from the dataset, reducing revenue by their optimized price, until the remaining revenue matches the current total. I counted the number of customers removed to determine the break-even churn count (how many of these customers would need to churn for the investment to not make sense). Finally, I calculated the revenue difference between the optimized and current revenue and the break-even churn rate for the selected segment.

In [14]:
df_filtered = df[df["churn_risk_quartiles"].isin(["Low"])].copy()
total_current_revenue = df_filtered["recognized_mrr"].sum()
total_optimized_revenue = df_filtered["optimized_price"].sum()

df_filtered = df_filtered.sort_values(by="new_churn_risk_score", ascending=False)

cumulative_churned = 0
cumulative_new_revenue = total_optimized_revenue

for index, row in df_filtered.iterrows():
    if cumulative_new_revenue <= total_current_revenue:
        break  
    
    cumulative_new_revenue -= row["optimized_price"]
    cumulative_churned += 1


revenue_difference = total_optimized_revenue - total_current_revenue

print(f"Total Current Revenue (Low Risk): {total_current_revenue:,.2f}")
print(f"Total Optimized Revenue (Low Risk / Before Churn): {total_optimized_revenue:,.2f}")
print(f"Revenue Increase (Optimized - Current): {revenue_difference:,.2f}")
print(f"Break-Even Churn Count (Low Risk): {cumulative_churned} customers")
print(f"Break-Even Churn Rate (Low Risk): {cumulative_churned / len(df_filtered):.2%}")

Total Current Revenue (Low Risk): 11,727.23
Total Optimized Revenue (Low Risk / Before Churn): 14,905.03
Revenue Increase (Optimized - Current): 3,177.80
Break-Even Churn Count (Low Risk): 27 customers
Break-Even Churn Rate (Low Risk): 21.60%


In [15]:
total_current_revenue = df["recognized_mrr"].sum()
total_optimized_revenue = df["optimized_price"].sum()
total_revenue_increase = df["optimized_price"].sum() - df["recognized_mrr"].sum()
total_revenue_change_perc = ((total_optimized_revenue / total_current_revenue) - 1)*100

print(f"Total Current Revenue: {total_current_revenue:,.2f}")
print(f"Total Optimized Revenue: {total_optimized_revenue:,.2f}")
print(f"Total Revenue Increase (Optimized - Current): {total_revenue_increase:,.2f}")
print(f"Total Revenue Change in %: {total_revenue_change_perc:,.2f}")

Total Current Revenue: 50,620.22
Total Optimized Revenue: 53,798.01
Total Revenue Increase (Optimized - Current): 3,177.80
Total Revenue Change in %: 6.28


### Final Remarks
After applying the optimized pricing model, total revenue increased from 50,620.22 to 54,145.40, a 6.96% increase. 

Among customers in the Low churn risk segments, revenue before churn increased from 11,727.23 to 15,252.42,  

To maintain the original revenue level after churn (optimized revenue = current revenue), 30 customers from Low Risk Customers (24% churn rate within these segments) would need to leave.

In [16]:
df.head(1)

,customer_id,cohort,price,price_name,commercial_type,invoicing_period_length,next_invoice_at,avg_3m_admin_bookings,avg_3m_user_bookings,city,...,normalized_admin_bookings,normalized_user_bookings,normalized_revenue,engagement_score,price_usage_fit,churn_risk_score,churn_risk_quartiles,normalized_engagmnet,optimized_price,new_churn_risk_score
0,399,01/07/2024,99,GoldPro 99€ monthly,jameda_gold_pro,1,28/02/2025 00:00,0,15,Hessen,...,0.0,0.041322,0.505155,0.051653,8.193527,16.361228,High,0.031477,99.0,16.361228


In [17]:
df_copy = df[[
    "customer_id", "cohort", "recognized_mrr", "price_name", "avg_3m_admin_bookings", "avg_3m_user_bookings",
    "churn_risk_score", "churn_risk_quartiles",
    "optimized_price", "new_churn_risk_score"
]].copy()

In [18]:
df_copy.to_csv("./My Projects/calc_results_table.csv", index = False)
df.to_csv("./My Projects/full_data_table.csv", index = False)

### Additional Useful Information Preperation

In [19]:
df.sort_values(by="normalized_revenue", ascending = True).sample(1)

,customer_id,cohort,price,price_name,commercial_type,invoicing_period_length,next_invoice_at,avg_3m_admin_bookings,avg_3m_user_bookings,city,...,normalized_admin_bookings,normalized_user_bookings,normalized_revenue,engagement_score,price_usage_fit,churn_risk_score,churn_risk_quartiles,normalized_engagmnet,optimized_price,new_churn_risk_score
472,277,01/07/2020,1188,Jameda import jameda_gold_pro 1188.00 m12,jameda_gold_pro,12,08/07/2025 00:00,85,8,Nürnberg,...,0.109677,0.022039,0.505155,0.137226,3.43116,6.793707,Medium,0.083625,99.0,6.793707


In [20]:
thresholds = df['churn_risk_score'].quantile([0.25, 0.50, 0.75]).tolist()
q1, q2, q3 = thresholds

def assign_risk_category(score, q1, q2, q3):
    if score <= q1:
        return 'Low'
    elif score <= q2:
        return 'Medium'
    elif score <= q3:
        return 'High'
    else:
        return 'Critical'

df['new_churn_risk_segments'] = df['new_churn_risk_score'].apply(lambda x: assign_risk_category(x, q1, q2, q3))

In [21]:
df["price_change_%"] = ((df["optimized_price"] / df["recognized_mrr"]) - 1)*100